# Schultze's Method of Voting
This notebook implements Schultze's method of voting 

In [ ]:
import numpy as np
import pandas as pd

## Test Data
Test data from a published Google sheet.

In [ ]:
test1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1591363425&single=true&output=csv'
test2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=541601874&single=true&output=csv'
test3 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=2108916501&single=true&output=csv'
test4 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=662509174&single=true&output=csv'
example1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=721130038&single=true&output=csv'
example2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1181234950&single=true&output=csv'
example3 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1865810740&single=true&output=csv'
example4 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=920600198&single=true&output=csv'
example6 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=753260081&single=true&output=csv'
example7 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=959087225&single=true&output=csv'

In [ ]:
def import_votes(url, first_row='voters'):
    if first_row=='voters':
        votes = pd.read_csv(url)
        #votes.columns = [(f'B{n:02d}') for n in range(votes.shape[1])]
    elif first_row=='count':
        vote_counts = pd.read_csv(url, header=None)
        votes = np.tile(vote_counts.iloc[1:,0].values, [np.int64(vote_counts.iloc[0,0]),1])
        if vote_counts.shape[1]>1:
            for i in np.arange(1,vote_counts.shape[1]):
                votes = np.append(votes, np.tile(vote_counts.iloc[1:,i].values, [np.int64(vote_counts.iloc[0,i]),1]), axis=0)
        votes = pd.DataFrame(votes.T)
    else:
        raise ValueError('Pass voters or count as first_row.')
    return votes

In [ ]:
votes = import_votes(example7, first_row='count')
votes

In [ ]:
candidates = np.unique(votes.values.reshape(votes.shape[0]*votes.shape[1]))
candidates

In [ ]:
len(candidates)

** randomize the candidate list here ?? **

## Preference Matrix
The preference matrix is the number of ballots that have candidate A over candidate B.

In [ ]:
def get_pref_count(votes, candidate_a, candidate_b):
    count = 0
    for i in range(votes.shape[1]):
        try:
            apos = votes.iloc[:,i][votes.iloc[:,i]==candidate_a].index.values[0]
        except:
            apos = None
        try:
            bpos = votes.iloc[:,i][votes.iloc[:,i]==candidate_b].index.values[0]
        except:
            bpos = None
            
        if apos == None:
            continue
        if bpos == None:
            if apos != None:
                count += 1
                continue
        if apos < bpos:
            count +=1
    return count

In [ ]:
def get_pref_matrix(votes):
    candidates = np.unique(votes.values.reshape(votes.shape[0]*votes.shape[1]))
    pref_vals = np.zeros([len(candidates), len(candidates)], dtype=np.int64)
    i = 0
    j = 0
    for candidate_a in candidates:
        for candidate_b in candidates:
            pref_vals[i,j] = get_pref_count(votes, candidate_a, candidate_b)
            j += 1
        i += 1
        j = 0
    return pd.DataFrame(pref_vals, columns=candidates, index=pd.Index(candidates))

In [ ]:
pref_matrix = get_pref_matrix(votes)
pref_matrix

## Path Strengths

In [ ]:
pref_vals = pref_matrix.values
p = pref_vals * 0
for i in range(len(candidates)):
    for j in range(len(candidates)):
        if pref_vals[i,j] > pref_vals[j,i]:
            p[i,j] = pref_vals[i,j]

for i in range(len(candidates)):
    for j in range(len(candidates)):
        if i != j:
            for k in range(len(candidates)):
                if i != j and j != k:
                    p[j,k] = np.max([p[j,k], min([p[j,i], p[i,k]])])

In [ ]:
p

## Schultze Winner Matrix

In [ ]:
winner = p*0
for i in range(len(candidates)):
    for j in range(len(candidates)):
        if i != j:
            if p[i,j] > p[j,i]:
                winner[i,j] = 1

In [ ]:
winner

## Overall Schultze Ranking

In [ ]:
ranking = pd.Series(np.sum(winner, axis=1), index=candidates).sort_values(ascending=False)

In [ ]:
ranking